In [1]:
import pandas as pd 
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("/kaggle/input/cleaned-data-17lacs-dropped-samecorr1/droppedna_highcorrdropped_merged_file.csv",encoding="utf-8")

In [3]:
df.head(n=2)

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,0,1,2,3,4,5,6,...,72,73,74,75,76,77,78,79,80,81
0,0,0,0,2541497,56,26325,22294,9018,25604,8390,...,1.713364,0.000000,0.115692,1.156922,0.269948,0.0,0.000000,0.0,0,0
1,1,1,1,2541785,50,26325,22294,9018,25604,12831,...,0.571121,0.571121,0.115692,1.002666,0.269948,0.0,37.384575,0.0,1,0


In [4]:
df.drop(["Unnamed: 0.2","Unnamed: 0","Unnamed: 0.1"],axis=1,inplace=True)

In [5]:
print(df.isnull().sum().any()>=1)

False


In [6]:
df.head(n=2)

,0,1,2,3,4,5,6,7,8,9,...,72,73,74,75,76,77,78,79,80,81
0,2541497,56,26325,22294,9018,25604,8390,27941,21218,869,...,1.713364,0.000000,0.115692,1.156922,0.269948,0.0,0.000000,0.0,0,0
1,2541785,50,26325,22294,9018,25604,12831,27941,21218,21533,...,0.571121,0.571121,0.115692,1.002666,0.269948,0.0,37.384575,0.0,1,0


In [7]:
df = df.loc[:,"2":"81"]
df.drop(["80"],axis=1,inplace=True)

In [8]:
df.columns

Index(['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14',
       '15', '16', '17', '18', '19', '20', '21', '22', '23', '26', '30', '31',
       '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43',
       '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55',
       '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67',
       '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79',
       '81'],
      dtype='object')

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:,-1:], test_size=0.01)

In [10]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Set the parameters for XGBoost
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'eta': 0.05,
    'max_depth': 100,
    'tree_method': 'gpu_hist',  # Use GPU for tree building
    'gpu_id': 0  # GPU device index
}

In [11]:
# Create DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [12]:
from tqdm import tqdm

In [13]:
# Train the model
model = xgb.train(params, dtrain, num_boost_round=100)

In [14]:
# Make predictions on the test set
y_pred = model.predict(dtest)


In [15]:
y_pred

array([0.02235937, 0.29520363, 0.1203369 , ..., 0.00753377, 0.87002087,
       0.01315094], dtype=float32)

In [16]:
y_pred.max(),y_pred.min()

(0.9963905, 0.0039743795)

In [17]:
from sklearn.metrics import log_loss

In [18]:
logloss = log_loss(y_test, y_pred, labels=[0, 1], normalize=True)
print("Log Loss: %.4f" % logloss)

Log Loss: 0.3196


In [19]:
y_pred = [1 if pred > 0.5 else 0 for pred in y_pred]

In [20]:
# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 86.86%


In [21]:
to_do_sheet = pd.read_csv('/kaggle/input/recsys/sharechat_recsys2023_data/T_est/000000000000.csv',encoding='utf-8')

In [22]:
to_do_sheet.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,70,71,72,73,74,75,76,77,78,79
0,0,64505,67,26325,7152,21563,19475,31440,27941,21621,...,1.519085,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,1,64506,67,20095,563,22861,19475,21280,27941,19203,...,0.000000,0.0,0.000000,0.0,0.115692,1.156922,0.269948,0.0,0.0,0.0
2,2,64507,67,890,22294,18294,21545,20210,27941,18800,...,0.982995,0.0,0.000000,0.0,0.000000,1.156922,0.269948,0.0,0.0,0.0
3,3,64508,67,20095,563,22861,25604,21280,27941,18800,...,0.000000,0.0,2.284486,0.0,0.000000,1.156922,0.269948,0.0,0.0,0.0
4,4,64509,67,27426,22294,11338,19475,23855,27941,21218,...,0.986040,0.0,0.000000,0.0,0.077128,0.077128,0.077128,0.0,0.0,0.0


In [23]:
row_numbers = to_do_sheet['0']

In [24]:
rows_with_nan = to_do_sheet.isnull().any(axis=1)
indexes_with_nan = to_do_sheet[rows_with_nan].index
print(indexes_with_nan)

Int64Index([     0,      2,      5,      6,     14,     15,     16,     17,
                27,     28,
            ...
            160937, 160940, 160941, 160944, 160953, 160958, 160959, 160962,
            160964, 160967],
           dtype='int64', length=51128)


In [25]:
to_do_sheet.fillna(to_do_sheet.median(),inplace=True)

In [26]:
to_do_sheet.drop(["Unnamed: 0","24","25","27","28","29"],axis=1,inplace=True)

In [27]:
to_do_sheet.drop(["0","1"],axis=1,inplace=True)

In [28]:
to_do_sheet.head()

,2,3,4,5,6,7,8,9,10,11,...,70,71,72,73,74,75,76,77,78,79
0,26325,7152,21563,19475,31440,27941,21621,14659,21574,31460,...,1.519085,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,20095,563,22861,19475,21280,27941,19203,14659,21574,27957,...,0.000000,0.0,0.000000,0.0,0.115692,1.156922,0.269948,0.0,0.0,0.0
2,890,22294,18294,21545,20210,27941,18800,9638,19343,30375,...,0.982995,0.0,0.000000,0.0,0.000000,1.156922,0.269948,0.0,0.0,0.0
3,20095,563,22861,25604,21280,27941,18800,14659,22970,31863,...,0.000000,0.0,2.284486,0.0,0.000000,1.156922,0.269948,0.0,0.0,0.0
4,27426,22294,11338,19475,23855,27941,21218,9638,21574,30778,...,0.986040,0.0,0.000000,0.0,0.077128,0.077128,0.077128,0.0,0.0,0.0


In [29]:
get_D_tds = xgb.DMatrix(to_do_sheet, label=None)

In [30]:
submitting_list = model.predict(get_D_tds)

In [31]:
submitting_list

array([0.34215564, 0.63896924, 0.04644407, ..., 0.00892155, 0.14877419,
       0.09579473], dtype=float32)

In [32]:
final_file = pd.DataFrame({'row_id':row_numbers,'is_clicked':submitting_list,'is_installed':submitting_list})

In [33]:
final_file.head()

,row_id,is_clicked,is_installed
0,64505,0.342156,0.342156
1,64506,0.638969,0.638969
2,64507,0.046444,0.046444
3,64508,0.731130,0.731130
4,64509,0.085080,0.085080


In [34]:
final_file.to_csv('new_file_xg.tsv', sep='\t', index=False)